# Cross-approximation

Often, we would like to build a $N$-dimensional tensor from a **black-box function** $f: \Omega \subset \mathbb{R}^N \to \mathbb{R}$, where $\Omega$ is a tensor product grid. That is, we are free to sample *whatever entries we want* within our domain $\Omega$, but we cannot afford to sample the entire domain (as it contains an **exponentially large number of points**). One way to build such a tensor is using *cross-approximation* ([I. Oseledets, E. Tyrtyshnikov: "TT-cross Approximation for Multidimensional Arrays"](http://www.mat.uniroma2.it/~tvmsscho/papers/Tyrtyshnikov5.pdf)) from well-chosen fibers in the domain.

We support two major use cases of cross-approximation in the TT format.

## Approximating a Function over a Domain

This is the more basic setting. We just need to specify:

- Our function of interest
- The tensor product domain $\Omega = u_1 \otimes \dots \otimes u_N$
- The number of TT ranks

In [1]:
import tntorch as tn
import torch

def function(x, y, z, t, w):  # Input arguments are vectors
    return 1 / (x + y + z + t + w)  # Hilbert tensor

domain = [torch.arange(1, 33) for n in range(5)]
t = tn.cross(function=function, domain=domain, ranks_tt=6)

print(t)

Cross-approximation over a 5D domain containing 3.35544e+07 grid points:
iter: 0 | eps: 1.068e-04 | total time:   0.0162
iter: 1 | eps: 7.863e-05 | total time:   0.0318
iter: 2 | eps: 7.395e-05 | total time:   0.0480
iter: 3 | eps: 7.461e-05 | total time:   0.0647
iter: 4 | eps: 7.199e-05 | total time:   0.0854
iter: 5 | eps: 7.461e-05 | total time:   0.1065 <- converged
Did 43968 function evaluations, which took 0.002092s (2.101e+07 evals/s)

5D TT tensor:

 32  32  32  32  32
  |   |   |   |   |
 (0) (1) (2) (3) (4)
 / \ / \ / \ / \ / \
1   6   6   6   6   1



Sometimes it's more convenient to work with functions that accept matrices (instead of a list of vectors) as input. We can do this with the `function_arg='matrix'` flag:

In [2]:
def function(Xs):  # Matrix (one row per sample, one column per input variable) and return a vector with one result per sample
    return 1/torch.sum(Xs, dim=1)

t = tn.cross(function=function, domain=domain, ranks_tt=6, function_arg='matrix')

Cross-approximation over a 5D domain containing 3.35544e+07 grid points:
iter: 0 | eps: 6.688e-05 | total time:   0.0309
iter: 1 | eps: 8.202e-05 | total time:   0.0527
iter: 2 | eps: 8.148e-05 | total time:   0.0733 <- converged
Did 22080 function evaluations, which took 0.003524s (6.266e+06 evals/s)



## Element-wise Operations on Tensors

Here we have one (or several) $N$-dimensional tensors that we want to transform element-wise. For instance, we may want to square each element of our tensor:

In [3]:
t2 = tn.cross(function=lambda x: x**2, tensors=t, ranks_tt=6)

Cross-approximation over a 5D domain containing 3.35544e+07 grid points:
iter: 0 | eps: 2.089e-04 | total time:   0.0138
iter: 1 | eps: 7.237e-04 | total time:   0.0267
iter: 2 | eps: 9.568e-04 | total time:   0.0397 <- converged
Did 22080 function evaluations, which took 0.0002475s (8.922e+07 evals/s)



Just for practice, let's do this now in a slightly different way by passing two tensors:

In [4]:
t2 = tn.cross(function=lambda x, y: x*y, tensors=[t, t], ranks_tt=6)

Cross-approximation over a 5D domain containing 3.35544e+07 grid points:
iter: 0 | eps: 3.017e-04 | total time:   0.0152
iter: 1 | eps: 6.855e-04 | total time:   0.0296
iter: 2 | eps: 7.822e-04 | total time:   0.0445 <- converged
Did 22080 function evaluations, which took 0.0002387s (9.252e+07 evals/s)



Let's check the accuracy of our cross-approximated squaring operation, compared to the groundtruth `t*t`:

In [5]:
tn.relative_error(t*t, t2)

tensor(0.0007)